# Create Initial Segmentations with TotalSegmentator v2 and MRSegmentator
The four dixon sequences of each section should share almost identical voxel positions within one examination. Therfore it should be sufficient to segment one sequence and directly map the results to the other three. I choose water only sequences for this task, as both models seem to perform best on them.
I only create mappings for section 1, 2 and 3, where most abdomibnal and chest organs/structures are located. Section 0, 4, 5 will be ignored.

I focus on 40 labels plus background:
- 20 labels of MRSegmentator
- heart
- urinary bladder
- spine (no distiction of individual vertebraes)
- sacrum
- left & right lung (no distinction between lobes)
- left & right iliac venes
- left & right iliac arteries
- left & right glueteus minimus
- left & right glueteus medius
- left & right glueteus maximus
- left & right femur
- left & right hip

Remark on space utilazation:  Segmentation masks need much less space than images (about 1GB for all segementations of one model). We don't need to be too concerned with saving resources.

### Use TotalSegmentator v2 to predict masks (executed in an sbatch file)

```bash
conda activate totalsegmentator

for file in $(ls nifti/*/*[1-3]_W.nii.gz)
do
    IFS="/" read -ra NAMES <<< "$file"
    TotalSegmentator -i $file -o "preds_total/${NAMES[1]}_${NAMES[2]}" --ml
done 
```

### Use MRSegmentator to predict masks

```bash
# filter for necessary files
nifti=$PATH1
scratch=$PATH2
[ -d $scratch ] && rm -r $scratch
mkdir $scratch
for file in $(ls $nifti/*/*[1-3]_W.nii.gz)
do
    IFS="/" read -ra NAMES <<< "$file"
    ln -s $file "$scratch"/"${NAMES[-2]}"_"${NAMES[-1]}"
done 


# activate conda environment
source /opt/conda/etc/profile.d/conda.sh
conda activate monai
cd $HOME/kidney/bundles/mr_segmentator


# start segmentation
python -m monai.bundle run inference \
  --meta_file configs/metadata.json \
  --config_file configs/inference.yaml \
  --logging_file configs/logging.conf \
  --dataset_dir $scratch
  

# clean
rm -r $scratch
```